In [1]:
import pandas as pd
import os
import time
import utils
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
DATA_DIR = "data/"
DATA_FILE_TRAIN = os.path.join(DATA_DIR, "bowfinal/train_bow.csv")
DATA_FILE_TEST = os.path.join(DATA_DIR, "bowfinal/test_bow.csv")

In [3]:
df = utils.csv_to_dataframe(DATA_FILE_TRAIN)

c = len(df.columns)

In [4]:
column_labels = list(range(0, c))
df.columns = column_labels
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,3615,3616,3617,3618,3619,3620,3621,3622,3623,3624
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [5]:
array = df.values
x = array[:,0:c-1]   # feature values
y = array[:,c-1]     # targets

#validation_size = 0.0000001
#seed = 11
#x_train, x_validation, y_train, y_validation = ms.train_test_split(x, y, test_size=validation_size, random_state=seed)

x_train = x
y_train = y

print(x.shape)
print(y.shape)
print('')
print(x_train.shape)
#print(x_validation.shape)
print('')
print(y_train.shape)


print(type(x_train))
print(type(y_train))

(7613, 3624)
(7613,)

(7613, 3624)

(7613,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


# Cross validation 5, and F1 Scoring#

In [8]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection as ms

num_folds = 5
seed = 11
scoring = 'f1'

kfold = ms.KFold(n_splits=num_folds, random_state=seed, shuffle=True)

# Multinomial Naive Bayes#

In [18]:
from sklearn.naive_bayes import MultinomialNB 

start = time.time()

parameters = {'alpha':[10,35,36]}
multinb = MultinomialNB()
MultiNBModel = (GridSearchCV(multinb, parameters)).fit(x_train, y_train)
scoreMultiNB = cross_val_score(MultiNBModel,  x_train, y_train, cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreMultiNB.mean(), "Parameters: ", MultiNBModel.best_params_, "Time Elapsed: ",  elapsed_time)

F1 Score:  0.7392902768706993 Parameters:  {'alpha': 35} Time Elapsed:  20.079333305358887


# Bernoulli Naive Bayes#

In [30]:
from sklearn.naive_bayes import BernoulliNB 

start = time.time()

parameters = {'alpha':[1.5]}
bernnb = BernoulliNB()
BernNBModel = (GridSearchCV(bernnb, parameters)).fit(x_train, y_train)
scoreBernNB = cross_val_score(BernNBModel,  x_train, y_train, cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreBernNB.mean(), "Parameters: ", BernNBModel.best_params_, "Time Elapsed: ",  elapsed_time)

F1 Score:  0.7622666002465467 Parameters:  {'alpha': 1.5} Time Elapsed:  11.592494010925293


# Linear SVC#

In [20]:
from sklearn.svm import LinearSVC

start = time.time()

parameters = { 'C':[0.01, 0.1, 1, 10], 'loss':('hinge', 'squared_hinge'),
              'multi_class':('ovr', 'crammer_singer') }
linearsvc = LinearSVC()
LinearSVCModel = (GridSearchCV(linearsvc, parameters)).fit(x_train, y_train)
scoreLinearSVC = cross_val_score(LinearSVCModel,  x_train, y_train, cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreLinearSVC.mean(), "Parameters: ", LinearSVCModel.best_params_, "Time Elapsed: ",  elapsed_time)

F1 Score:  0.7451768672088044 Parameters:  {'C': 0.01, 'loss': 'squared_hinge', 'multi_class': 'ovr'} Time Elapsed:  144.63518643379211


# K-nearest Neighbors #

In [50]:
from sklearn import neighbors

start = time.time()

parameters = {'n_neighbors':[5, 7, 11]}
knn = neighbors.KNeighborsClassifier()
KNNModel = (GridSearchCV(knn, parameters)).fit(x_train, y_train)
scoreKNN = cross_val_score(KNNModel,  x_train, y_train,cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreKNN.mean(), "Parameters: ", KNNModel.best_params_, "Time Elapsed: ",  elapsed_time)

F1 Score:  0.5671974483141675 Parameters:  {'n_neighbors': 5} Time Elapsed:  3086.262533187866


# Decision Tree Classifier #

In [42]:
from sklearn import tree

start = time.time()

parameters = {'min_samples_split':[2, 3 ,4]}
decisiontc =  tree.DecisionTreeClassifier()
DecisionTreeCModel = (GridSearchCV(decisiontc, parameters)).fit(x_train, y_train)
scoreDecisionTreeC = cross_val_score(DecisionTreeCModel,  x_train, y_train,cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreDecisionTreeC.mean(), "Parameters: ", DecisionTreeCModel.best_params_, "Time Elapsed: ",  
      elapsed_time)

F1 Score:  0.7049455066239487 Parameters:  {'min_samples_split': 3} Time Elapsed:  1331.8315510749817


# Random Forest Classifier #

In [21]:
from sklearn import ensemble

start = time.time()


randomfc1 =  ensemble.RandomForestClassifier(n_estimators  = 500)
randomForestC1Model = randomfc1.fit(x_train, y_train)
scorerandomForestC1 = cross_val_score(randomForestC1Model,  x_train, y_train,cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scorerandomForestC1.mean(), "Time Elapsed: ",  elapsed_time)

F1 Score:  0.732365851254758 Parameters:  {'n_estimators': 100} Time Elapsed:  808.0385329723358


# AdaBoost Classifier #

In [22]:
start = time.time()

parameters = {'n_estimators':[50,75, 100]}
adaboost = ensemble.AdaBoostClassifier()
adaBoostModel = (GridSearchCV(adaboost, parameters)).fit(x_train, y_train)
scoreadaBoost = cross_val_score(adaBoostModel,  x_train, y_train,cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoreadaBoost.mean(), "Parameters: ", adaBoostModel.best_params_, "Time Elapsed: ",  elapsed_time)

F1 Score:  0.6884400442712162 Parameters:  {'n_estimators': 50} Time Elapsed:  1401.2611541748047


# Gradient Boosting Classifier #

In [23]:
start = time.time()

parameters = {'learning_rate':[1, 1.5,2], 'n_estimators':[50, 75, 100]}
gradientboostc = ensemble.GradientBoostingClassifier()
gradientBoostCModel = (GridSearchCV(gradientboostc, parameters)).fit(x_train, y_train)
scoregradientBoostC = cross_val_score(gradientBoostCModel,  x_train, y_train,cv= kfold, scoring = scoring)

elapsed_time = time.time() - start

print("F1 Score: ",  scoregradientBoostC.mean(), "Parameters: ", gradientBoostCModel.best_params_,
      "Time Elapsed: ",  elapsed_time)

F1 Score:  0.7251468470315793 Parameters:  {'learning_rate': 1, 'n_estimators': 50} Time Elapsed:  7661.863627910614


# KAGGLE TEST SET #

In [10]:
df_test = utils.csv_to_dataframe(DATA_FILE_TEST)
df_test.head(3)    

,kw_ablaze,kw_accident,kw_aftershock,kw_airplaneaccident,kw_ambulance,kw_annihilated,kw_annihilation,kw_apocalypse,kw_armageddon,kw_army,...,ûïa,ûïthe,ûïwe,ûïwhen,ûò,ûó,û÷extrem,û÷polit,åè,åê
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
column_labels = list(range(0, c-1))
df_test.columns = column_labels

In [12]:
array_test = df_test.values
x_test = array_test[:,0:c]   # feature values
print(type(x_test))
print(x_test.shape)

<class 'numpy.ndarray'>
(3263, 3624)


# Bern NB PREDICTIONS#

In [59]:
bowBernModel =  BernoulliNB(alpha =  3.75)
bowBernModel.fit(x_train, y_train)

predictions = bowBernModel.predict(x_test)
predictions_list = np.array(predictions).tolist()

print(type(predictions_list))
print(len(predictions_list))
print(predictions_list[0:10])

<class 'list'>
3263
[1, 1, 1, 1, 1, 1, 0, 0, 0, 0]


# PREPARE SUBMISSION FILE#

In [60]:
DATA_FILE_TEST_ID = os.path.join(DATA_DIR, "test_id.csv")

test_id_list = utils.csv_to_list_of_strings(DATA_FILE_TEST_ID)

print(len(test_id_list))
print(test_id_list[0:10])

3263
['0', '2', '3', '9', '11', '12', '21', '22', '27', '29']


In [61]:
df_test_id = utils.csv_to_dataframe(DATA_FILE_TEST_ID)
df_test_id.shape

(3263, 1)

In [62]:
df_test_predict = pd.DataFrame({'col':predictions_list})
df_test_predict.columns = ['target']
df_test_predict.shape

(3263, 1)

In [63]:
df_submit = pd.concat([df_test_id, df_test_predict], axis=1)
df_submit.head(10)

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
5,12,1
6,21,0
7,22,0
8,27,0
9,29,0


In [64]:
DATA_FILE_SUBMIT = os.path.join(DATA_DIR, "submissionBernBow375.csv")

utils.dataframe_to_csv(df_submit, DATA_FILE_SUBMIT)